In [1]:
from pyomo.environ import *
import os
model = ConcreteModel()
solverexe = "gurobi"
infinity = float('inf')
# dirsolver = r"C:\Users\ch9fod\Documents\GitHub\ED\solvers"
# datafile = "data1.dat"

In [2]:
#set
model.Bus = RangeSet(4, doc='Buses')
model.G = RangeSet(3, doc='Generators')

In [3]:
#parameters
model.a = Param(model.G, initialize={1:20 , 2:25 , 3:40}, 
                doc='Parameter a')
model.b = Param(model.G, initialize={1:0.05 , 2:0.10 , 3:0.20}, 
                doc='Parameter b')
model.Pmin = Param(model.G, initialize={1:20 , 2:20 , 3:30}, 
                   doc='Parameter Pmin')
model.Pmax = Param(model.G, initialize={1:400 , 2:300 , 3:250}, 
                   doc='Parameter Pmax')
model.Loads = Param(model.Bus, initialize={1:0 , 2:0 , 3:200 , 4:400}, 
                    doc='Loads for each Bus')
#lone parameter, constant
model.D = Param(initialize=600, doc='Load')
model.BaseP = Param(initialize=100, doc='Base Power')
model.PFmax = Param(initialize=300, doc='Max Power Flow')
model.PFmin = Param(initialize=-300, doc='Min Power Flow')
dtab = {
    (1,2) : 0.007,
    (2,3) : 0.01,
    (3,4) : 0.006,
    (4,1) : 0.05,    
    (2,1) : 0.007,
    (3,2) : 0.01,
    (4,3) : 0.006,
    (1,4) : 0.05,
    }
model.X = Param(model.Bus, model.Bus, initialize=dtab, 
                doc='Reactance between buses', default=infinity)

In [4]:
#variables
model.P = Var(model.G, doc='Generation per Generator')
model.Angles = Var(model.Bus, doc='Angles of Buses')

In [5]:
#constraints
def maxp(model,i):
    return model.P[i] <= model.Pmax[i]
model.maxprod = Constraint(model.G, rule = maxp)

def minp(model,i):
    return model.P[i] >= model.Pmin[i]
model.minprod = Constraint(model.G, rule = minp)

def demand_r(model):
    return model.D == sum(model.P[i] for i in model.G)
model.demand = Constraint(model.G, rule = demand_r)

def slackRule(model):
    return model.Angles[1] == 0
model.slackBus = Constraint(rule=slackRule)

def ruleBus1(model):
    return (model.P[1] - model.Loads[1])/model.BaseP == \
            ((1/model.X[1,2]) * (model.Angles[1]-model.Angles[2]) + \
             (1/model.X[1,4]) * (model.Angles[1]-model.Angles[4]))
model.Bus1 = Constraint(rule=ruleBus1)

def ruleBus2(model):
    return (model.P[2] - model.Loads[2])/model.BaseP == \
            ((1/model.X[2,1]) * (model.Angles[2]-model.Angles[1]) + \
             (1/model.X[2,3]) * (model.Angles[2]-model.Angles[3]))
model.Bus2 = Constraint(rule=ruleBus2)

def ruleBus3(model):
    return (model.P[3] - model.Loads[3])/model.BaseP == \
            ((1/model.X[3,2]) * (model.Angles[3]-model.Angles[2]) + \
             (1/model.X[3,4]) * (model.Angles[3]-model.Angles[4]))
model.Bus3 = Constraint(rule=ruleBus3)

def ruleBus4(model):
    return (-model.Loads[4])/model.BaseP == \
            ((1/model.X[4,3]) * (model.Angles[4]-model.Angles[3]) + \
             (1/model.X[4,1]) * (model.Angles[4]-model.Angles[1]))
model.Bus4 = Constraint(rule=ruleBus4)

def ruleFlow12(model):
    return (model.PFmin/model.BaseP , \
            (1/model.X[1,2])*(model.Angles[1]-model.Angles[2]) , \
            model.PFmax/model.BaseP)
model.Flow12 = Constraint(rule=ruleFlow12)

def ruleFlow23(model):
    return (model.PFmin/model.BaseP , \
            (1/model.X[2,3])*(model.Angles[2]-model.Angles[3]) , \
            model.PFmax/model.BaseP)
model.Flow23 = Constraint(rule=ruleFlow23)

def ruleFlow34(model):
    return (model.PFmin/model.BaseP , \
            (1/model.X[3,4])*(model.Angles[3]-model.Angles[4]) , \
            model.PFmax/model.BaseP)
model.Flow34 = Constraint(rule=ruleFlow34)

def ruleFlow41(model):
    return (model.PFmin/model.BaseP , \
            (1/model.X[4,1])*(model.Angles[4]-model.Angles[1]) , \
            model.PFmax/model.BaseP)
model.Flow41 = Constraint(rule=ruleFlow41)

In [6]:
#objective
def cost_rule(model):
    return sum(model.a[i]*model.P[i] + 
               0.5*model.b[i]*model.P[i]**2 
               for i in model.G)
#default is to minimize        
model.OBJ = Objective(rule=cost_rule) 

In [7]:
if solverexe == "gurobi":
    solver = SolverFactory(solverexe)   
else:
    solver = SolverFactory(solverexe, 
                           executable=os.path.join(dirsolver, solverexe))
instance = model.create_instance()
instance.dual = Suffix(direction=Suffix.IMPORT)
results = solver.solve(instance)

	constructed model; returning a clone of the current model instance.


In [8]:
instance.display()
instance.pprint()

Model unknown

  Variables:
    P : Generation per Generator
        Size=3, Index=G
        Key : Lower : Value         : Upper : Fixed : Stale : Domain
          1 :  None :   347.6861167 :  None : False : False :  Reals
          2 :  None : 76.6599597587 :  None : False : False :  Reals
          3 :  None : 175.653923541 :  None : False : False :  Reals
    Angles : Angles of Buses
        Size=4, Index=Bus
        Key : Lower : Value            : Upper : Fixed : Stale : Domain
          1 :  None :              0.0 :  None : False : False :  Reals
          2 :  None : -0.0156338028169 :  None : False : False :  Reals
          3 :  None : -0.0456338028169 :  None : False : False :  Reals
          4 :  None : -0.0621730382294 :  None : False : False :  Reals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 22297.786720300675

  Constraints:
    maxprod : Size=3
        Key : Lower : Body          : Upper
          1 :

In [9]:
value(instance.OBJ)

22297.786720300675

In [10]:
p = [0, 0, 0 ,0]
for i in range(3):
    p[i+1] = value(instance.P[i+1])

In [11]:
print ("Duals")
from pyomo.core import Constraint
for c in instance.component_objects(Constraint, active=True):
    print ("   Constraint",c)
    cobject = getattr(instance, str(c))
    for index in cobject:
        print ("      ", index, instance.dual[cobject[index]])

Duals
   Constraint maxprod
       1 -0.0
       2 -0.0
       3 -0.0
   Constraint minprod
       1 0.0
       2 0.0
       3 0.0
   Constraint demand
       1 -0.0
       2 32.6659959759
       3 -0.0
   Constraint slackBus
       None -0.0
   Constraint Bus1
       None 471.830985916
   Constraint Bus2
       None -0.0
   Constraint Bus3
       None 4246.47887324
   Constraint Bus4
       None -3842.05231388
   Constraint Flow12
       None -0.0
   Constraint Flow23
       None -4920.52313883
   Constraint Flow34
       None -4.24183760303e-10
   Constraint Flow41
       None -0.0
